# Wide&Deep

Wide&Deep 是 Google 在 2016 年发表于 DLRS 上的论文中提出来的，结合了 Wide 模型的 `Memorization` 和 Deep 模型的 `Generalization`。

![Wide & Deep](imgs/W&D.jpg)

如上图所示，W&D 由两部分组成：Wide 和 Deep。Wide 是一个广义的线性模型，用于捕捉在历史数据中出现过的模式 / 特征规律。Deep 是一个前馈神经网络，通过稀疏的类别特征学习特征稠密、低低维的 embedding，有更好的泛化性能。

Wide 是一个广义的线性模型，一般是逻辑回归模型：
$$
y_{wide} = \boldsymbol{w}^T \boldsymbol{x} + b,\ \boldsymbol{x} \in \mathbb{R}^d
$$
并且 Wide 部分可以加入人工构造的特征，即人工设计交叉特征，如交叉特征 $\phi_k$:
$$
\phi_K(\boldsymbol{x}) = \prod_{i=1}^d x_i^{c_{ki}}
$$
其中 $c_{ki}$ 表示第 $i$ 个特征 $x_i$ 是否出现在交叉特征 $\phi_k$ 中。

Deep 是一个前馈神经网络，一般是由多层神经网络组成。
$$
y_{deep} = DNN(\boldsymbol{x})
$$

最终二者相加后经过 `sigmoid` 激活函数得到最终的输出：
$$
P(Y = 1 | \boldsymbol{x}) = \hat{y} = \sigma(\boldsymbol{w}_{wide}^T [\boldsymbol{x}, \phi(\boldsymbol{x})] + \boldsymbol{w}^T_{deep} \boldsymbol{a}^{l_f} + b)
$$

其实从这个式子我们也可以看出，Wide&Deep 将输入 sigmoid 之前的回归值拆成了两部分，一部分来自 Wide 部分，一部分来自 Deep 部分。Wide 和 Deep 是联合训练的，通过梯度下降来更新参数，

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import itertools
from tqdm import tqdm
from collections import namedtuple

import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import *

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import  MinMaxScaler, LabelEncoder

from collections import namedtuple

In [6]:
data_path = "./data/criteo_sample.txt"

In [7]:
dataset = pd.read_csv(data_path)
dataset

,label,I1,I2,I3,I4,I5,I6,I7,I8,I9,...,C17,C18,C19,C20,C21,C22,C23,C24,C25,C26
0,0,NaN,3,260.0,NaN,17668.0,NaN,NaN,33.0,NaN,...,e5ba7672,87c6f83c,NaN,NaN,0429f84b,NaN,3a171ecb,c0d61a5c,NaN,NaN
1,0,NaN,-1,19.0,35.0,30251.0,247.0,1.0,35.0,160.0,...,d4bb7bd8,6fc84bfb,NaN,NaN,5155d8a3,NaN,be7c41b4,ded4aac9,NaN,NaN
2,0,0.0,0,2.0,12.0,2013.0,164.0,6.0,35.0,523.0,...,e5ba7672,675c9258,NaN,NaN,2e01979f,NaN,bcdee96c,6d5d1302,NaN,NaN
3,0,NaN,13,1.0,4.0,16836.0,200.0,5.0,4.0,29.0,...,e5ba7672,52e44668,NaN,NaN,e587c466,NaN,32c7478e,3b183c5c,NaN,NaN
4,0,0.0,0,104.0,27.0,1990.0,142.0,4.0,32.0,37.0,...,e5ba7672,25c88e42,21ddcdc9,b1252a9d,0e8585d2,NaN,32c7478e,0d4a6d1a,001f3601,92c878de
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,0,NaN,0,113.0,3.0,3036.0,575.0,2.0,3.0,214.0,...,07c540c4,9880032b,21ddcdc9,5840adea,34cc61bb,c9d4222a,32c7478e,e5ed7da2,ea9a246c,984e0db0
196,1,0.0,1,1.0,1.0,1607.0,12.0,1.0,12.0,15.0,...,1e88c74f,3972b4ed,NaN,NaN,d1aa4512,NaN,32c7478e,9257f75f,NaN,NaN
197,1,1.0,0,6.0,3.0,0.0,0.0,19.0,3.0,3.0,...,3486227d,5aed7436,54591762,a458ea53,4a2c3526,NaN,32c7478e,1793a828,e8b83407,1a02cbe1
198,0,0.0,22,6.0,22.0,203.0,153.0,80.0,18.0,508.0,...,3486227d,13145934,55dd3565,5840adea,bf647035,NaN,32c7478e,1481ceb4,e8b83407,988b0775


# tensorflow 实现

参见：[WideNDeep-tf.py](./WideNDeep-tf.py)。该实现来自[这里](https://github.com/datawhalechina/fun-rec/blob/master/codes/base_models/WideNDeep.py)。

# pytorch 实现

# 总结
**一些问题：**
1. 为什么 Wide 侧会有记忆性？  
因为 Wide 侧的输入来源于类别特征，通常会对类别特征进行 one-hot 展开，而 Wide 侧是一个线性模型，会对不同特征有不同的权重，权重的大小反映了特征的强弱。在历史数据中出现过的一些特征组合会体现在权重的大小上，当再次出现类似的特征时，强的特征（组合）会获得更高的预测值，这就是 Wide 侧的记忆能力。Wide 部分学习的是历史数据中特征的共现频率，**如果一个特征组合于目标变量有很强的正相关，则该特征组合会获得较高的权重**。

2. 对于输入给 Wide 侧的数值特征，还会有记忆能力吗？  
xxx

3. 为什么推荐算法的网络层数都不高?   
网络层数太深会导致模型难以训练，误差反向传播过程中，越底层的网络训练难度越大。庞大的embedding层可能更加难以训练(好像有看到说对于embedding可能会采用不同的学习率，对于频繁更新的，学习率较小，而更新不频繁的，学习率较大）。推荐系统的网络并不是做不深，而是不能太深。原因有二，1) 深度的网络需要大量的信息来进行拟合的，而推荐系统的输入往往是一些高维度的稀疏特征，携带的信息有限，因此只需要较浅的网络就足够学习了，如果强行增加深度，反而网络不能收敛；2) 推荐系统往往是一个实时打分的系统，个人觉得，还是在保证效果的前提下，网络越简单越好，这样能保证请求延迟等各方面都处在比较好的程度。如果为了提高0.1%的效果，消耗了远远多与收益的成本，对于企业来说是不划算的。比如，xdeepfm，nffm，完全是堆积木。

4. Deep 侧的泛化能力是什么？
Deep 的泛化能力指的是对于历史数据中未出现的或者很少的特征组合也能给出正确的预测，即使**出现了新的特征组合也能够正确推断出其对目标变量的贡献**。为什么 Deep 部分能泛化呢？Deep 侧对类别特征进行了表征，利用历史数据发掘特征之间隐藏的关系。